In [1]:
import base64
import imageio
import IPython
import tensorflow as tf

2023-02-01 14:42:51.759388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 14:42:51.867587: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64${LD_LIBRARY_PATH:+:}
2023-02-01 14:42:51.867605: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-01 14:42:52.393898: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.

In [2]:
def train_agent(agent, dataset, collect_driver, tf_env, n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(tf_env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")

def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename, "rb").read()
    b64 = base64.b64encode(video)
    tag = """
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>""".format(
        b64.decode()
    )
    return IPython.display.HTML(tag)

def create_policy_eval_video(tf_env, env, agent, filename, num_episodes=5, fps=30):
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
        for _ in range(num_episodes):
            time_step = tf_env.reset()
            policy_state = agent.collect_policy.get_initial_state(tf_env.batch_size)
            action_step = agent.policy.action(time_step, policy_state)
            video.append_data(env.render())
            while not time_step.is_last():
                action_step = agent.policy.action(time_step, action_step.state)
                time_step = tf_env.step(action_step.action)
                video.append_data(env.render())
    return embed_mp4(filename)

In [3]:
def train():
    env = suite_gym.load("CartPole-v0")
    tf_env = tf_py_environment.TFPyEnvironment(env)
    # Network
    rnn_network = q_rnn_network.QRnnNetwork(
        tf_env.observation_spec(), tf_env.action_spec(), lstm_size=(20,)
    )
    # Agent
    update_period = 4  # train the model every 4 steps
    train_step = tf.Variable(0)
    agent = DqnAgent(
        tf_env.time_step_spec(),
        tf_env.action_spec(),
        q_network=rnn_network,
        optimizer=tf.keras.optimizers.Nadam(learning_rate=0.01),
        target_update_period=10,
        td_errors_loss_fn=tf.keras.losses.Huber(
            reduction="none"
        ),  # tf.keras.losses.binary_crossentropy
        gamma=0.95,  # discount factor
        train_step_counter=train_step,
        epsilon_greedy=lambda: tf.keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=1.0, decay_steps=2500, end_learning_rate=0.01
        )(train_step),
    )
    agent.initialize()
    # Replay buffer
    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        data_spec=agent.collect_data_spec,
        batch_size=tf_env.batch_size,
        max_length=10000,
    )
    replay_buffer_observer = replay_buffer.add_batch
    # Training metrics
    training_metrics = [
        tf_metrics.NumberOfEpisodes(),
        tf_metrics.EnvironmentSteps(),
        tf_metrics.AverageReturnMetric(),
        tf_metrics.AverageEpisodeLengthMetric(),
    ]
    # Collect driver
    collect_driver = DynamicStepDriver(
        tf_env,
        agent.collect_policy,
        observers=[replay_buffer_observer] + training_metrics,
        num_steps=update_period,  # train the model every 4 steps
    )
    initial_collect_policy = RandomTFPolicy(
        tf_env.time_step_spec(), tf_env.action_spec()
    )
    initial_driver = DynamicStepDriver(
        tf_env,
        initial_collect_policy,
        observers=[replay_buffer.add_batch],
        num_steps=1000,
    )
    final_time_step, final_policy_state = initial_driver.run()
    # The Dataset
    dataset = replay_buffer.as_dataset(
        sample_batch_size=64, num_steps=2, num_parallel_calls=3
    ).prefetch(3)
    # Training Loop
    collect_driver.run = function(collect_driver.run)
    agent.train = function(agent.train)
    return agent, dataset, collect_driver, tf_env, env

In [4]:
agent, dataset, collect_driver, tf_env, env = train()
train_agent(agent, dataset, collect_driver, tf_env, 10000)
create_policy_eval_video(tf_env, env, agent, "trained-agent", num_episodes=1)

NameError: name 'suite_gym' is not defined